# Random Allelic Expression in the Human Body
### Stephanie N. Kravitz, Aaron R. Quinlan, Christopher Gregg

#### This notebook provides the code to perform the binomial vs. beta-binomial hypothesis testing to identify random allelic expressed genes compared to biallelic expressed genes for a single GTEx individual. 

#### The analysis requires two dataframes, each with gene IDs as rows and tissues/samples as columns. One dataframe comprises of allele-specific expression counts ("Allele 1") for each gene, and the other file should consist of total counts (i.e. "Allele 1" + "Allele 2" counts). 

### First import necessary libraries:

In [7]:
## To run R code in jupyter notebook, first install the the rpy2 library.

## Run the following: 

# ! pip3 install rpy2


## Once rpy2 is installed, execute the following to load the library:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
%%R

## Libraries:
require(VGAM)
options(warn=-1)

library("qvalue")

R[write to console]: Loading required package: VGAM

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: splines



### Read in our data files of "Allele 1" and "Total" allelic expression counts:

In [44]:
%%R

## For the purpose of this exercise, we will just run the test on a few genes

## "Allele 1" counts:
file_y <- '../data/GTEX-YFC4/GTEX-YFC4_to-plot_counts.y.txt'
## "Total" counts:
file_tot <- '../data/GTEX-YFC4/GTEX-YFC4_to-plot_counts.tot.txt'

y <- read.table(file_y, sep="\t", header=T, row.names=1)
n <- read.table(file_tot, sep="\t", header=T, row.names=1)

#print(head(y))

## Drop GENE_NAME columns:
y <- subset(y, select = -c(GENE_NAME))
n <- subset(n, select = -c(GENE_NAME))

print(head(y))


                   OVARY SKINS LUNG BRNCDT ARTCRN VAGINA MSCLSK BRNSPC BRNCHA
ENSG00000100030.14   252   466  427   1427    422    290     99    605    417
ENSG00000185829.17    46    38   24     29     25     56     16     23     57
ENSG00000178184.15    97   339   33     32     70    133     NA     40     21
ENSG00000102081.13    64    34   61     49     50     45     14     47     28
                   BRNNCC HRTAA THYROID LIVER BRNPTM FIBRBLS NERVET PTTARY
ENSG00000100030.14    680   428     532   243   1156     460    424    356
ENSG00000185829.17     27    26      94    22     25      11     25     41
ENSG00000178184.15     38    13     101    35     24      20     60     70
ENSG00000102081.13     59    42     131    58     43      NA     37     71
                   BRNCHB ADPSBQ WHLBLD BRNCTXB ESPGEJ BRNSNG ARTTBL BRNHPT
ENSG00000100030.14    777    411    407     925    270    603    479    877
ENSG00000185829.17     47     26     19      22     47     14     22     14
ENSG000

### Function to perform the binomial and beta-binomial tests:

In [47]:
%%R 

## This function takes in: 
# allele counts (xx)
# total counts (nn)
# "do beta-binomial" --> whether to perform the beta-binomial test, or simulated 'null' hypothesis, 
# which is mostly for data QC purposes to make sure the test is working correctly
# "Simulate the Null Hypothesis" --> if True, this calculates the binomial distribution from the mean allele ratio (probability)

bb.test <- function(xx, nn, dobb=T, simH0=F) 
{
    x = xx[!is.na(xx)]
    #print(x)
    n = nn[!is.na(nn)]
    #print(n)
    p = sum(x)/sum(n)
    #print(p)
    

    # This option allows us to simulated under the null
    # hypothesis to check that the test is working correctly.
    if (simH0)
    {
      # This line replaces the observed data with # observations simulated under the null.
        x = rbinom(length(n),n,p)
    }

    ll0 = sum(log(dbinom(x,n,p)))
    df0 = 1

    if (dobb)
    {
        # Do the test with Beta Binomial alternative.
        fit = vglm(cbind(x,n-x) ~ 1, family=betabinomial)
        mu = Coef(fit)[1]
        rho = Coef(fit)[2]
        ll1 = sum(log(dbetabinom(x,n,mu,rho)))
        df1 = 2
    }

    else
    {
        # Do the test with free alternative.
        ll1 = sum(log(dbinom(x,n,x/n)))
        df1 = length(x)
    }
    
    fit <- pchisq(2*(ll1-ll0), df1-df0, lower.tail=F)
}

### Perform the test on each gene: 

In [50]:
%%R

gene_ids <- as.vector(row.names(y))

## OPTIONAL: perform the 'simulated null' biallelic data using the mean allele ratio (p) to fit the distribution
null_results <- list()
for (gene in gene_ids) {
    tissue_num <- ncol(y) - sum(is.na(y[gene, ]))

    fit = try(bb.test(y[gene, ], n[gene, ], dobb=T, simH0=T), TRUE)

    if(isTRUE(class(fit) == "try-error")) { next } else {

    null_results[[gene]] <- c(tissue_num, fit)
    }
}

null_fits <- do.call("rbind", null_results)
colnames(null_fits) <- c("tissue_num", "bb_null") # null hypothesis results


## For the actual beta-binomial test on the data as-is:
data_results <- list()
for (gene in gene_ids) {
    fit = try(bb.test(y[gene, ], n[gene, ], dobb=T, simH0=F), TRUE)

    if(isTRUE(class(fit) == "try-error")) { next } else {

    data_results[[gene]] <- c(fit)
    }
}

data_fits <- do.call("rbind", data_results)
colnames(data_fits)[1] <- "bb_test" # real/observed data hypothesis results

# merge both fits, by GENE_ID
all_fits <- merge(null_fits, data_fits, by = "row.names", all = TRUE)

colnames(all_fits) <- c("GENE_ID", "tissue_num", "bb_null", "bb_test")
print(all_fits)

             GENE_ID tissue_num    bb_null      bb_test
1 ENSG00000100030.14         34 0.94736136 1.000000e+00
2 ENSG00000102081.13         33 0.05608945 1.008926e-17
3 ENSG00000178184.15         32 0.04341246 5.070390e-06
4 ENSG00000185829.17         34 0.29410716 2.468206e-01


### Perform q-value correction on p-values for multiple-testing correction:

In [52]:
%%R

### Calculate q-values:
### NOTE: may not work on small numbers of genes. TODO: add file that has a couple thousand genes to assess

r <- all_fits

# Given a set of p-values, the qvalue object can be calculated by using the qvalue function:

# Null Simulations:
p_null <- r$bb_null
qobj_null <- qvalue(p = p_null)

# Observed Data:
p_data <- r$bb_test
qobj_data <- qvalue(p = p_data)

# Once the qvalue object is created, estimates of the q-values, the proportion of true null hypotheses, and the local false discovery rates can be accessed from qobj:

# Null Simulations:
null_qvalues <- qobj_null$qvalues # q-values
null_pi0 <- qobj_null$pi0 # proportion true null hypotheses
null_lfdr <- qobj_null$lfdr # local false discovery rates

# Observed Data:
data_qvalues <- qobj_data$qvalues # q-values
data_pi0 <- qobj_data$pi0 # proportion true null hypotheses
data_lfdr <- qobj_data$lfdr # local false discovery rates

r_qobj <- cbind(r, null_qvalues, null_lfdr, data_qvalues, data_lfdr)

print(r_qobj)

R[write to console]: Error in smooth.spline(lambda, pi0, df = smooth.df) : 
  missing or infinite values in inputs are not allowed




Error in smooth.spline(lambda, pi0, df = smooth.df) : 
  missing or infinite values in inputs are not allowed


RInterpreterError: Failed to parse and evaluate line '\n### Calculate q-values:\n\nr <- all_fits\n\n# Given a set of p-values, the qvalue object can be calculated by using the qvalue function:\n\n# Null Simulations:\np_null <- r$bb_null\nqobj_null <- qvalue(p = p_null)\n\n# Observed Data:\np_data <- r$bb_test\nqobj_data <- qvalue(p = p_data)\n\n# Once the qvalue object is created, estimates of the q-values, the proportion of true null hypotheses, and the local false discovery rates can be accessed from qobj:\n\n# Null Simulations:\nnull_qvalues <- qobj_null$qvalues # q-values\nprint(null_qvalues)\nnull_pi0 <- qobj_null$pi0 # proportion true null hypotheses\nnull_lfdr <- qobj_null$lfdr # local false discovery rates\n\n# Observed Data:\ndata_qvalues <- qobj_data$qvalues # q-values\ndata_pi0 <- qobj_data$pi0 # proportion true null hypotheses\ndata_lfdr <- qobj_data$lfdr # local false discovery rates\n\nr_qobj <- cbind(r, null_qvalues, null_lfdr, data_qvalues, data_lfdr)\n\nprint(r_qobj)\n